In [39]:
### parameters ###

n_cont = 2 # number of compartments for each state
dt=0.1 # time step size
L_days = 3 # length of latent period
I_days =4 # length of infectious period
dir = r"drive/My Drive/PhD/COVID19 Exit Strategy - Uri"
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from sklearn.linear_model import LinearRegression
from IPython.core.debugger import set_trace
import ipywidgets as widgets

def SEIR_erlang(X,n_groups,n_comp,Rt,L_days,I_days,delta,dt,rho,W=1.5,L=0.6):
  
  # X - matrix with N groups and M compartments
  # n_comp - number of compartments - k of Erlang
  # n_groups - number of shifts
  # Rt - vector of R values for each group
  # L_days - latent period
  # I_days - infectious period
  # delta - the fraction of spillover
  # Rf - the rate of infections from one group to the other
  # dt - time step
  # W - R for work days
  # L - R for lockdown days
  
  # if we are simulating only one group rho has to be equal to one
  if n_groups == 1:
    rho=1

  dSdt = np.zeros([n_groups])
  dEdt = np.zeros([n_groups,n_comp])
  dIdt = np.zeros([n_groups,n_comp])
  dRdt = np.zeros([n_groups])
  
  # run time step for each group
  for group in np.arange(n_groups):
    # load state from input
    x = X[group,:].squeeze()
    E = x[:n_comp]
    I = x[n_comp:2*n_comp]
    S = x[-2]
    R= x[-1]
    
    # define the number of infected people in other groups and their R
    I_other = np.delete(X,group,axis=0)[:,n_comp:2*n_comp].sum(axis=1)
    R_other = np.delete(Rt,group,axis=0)
    
    
    d=0.01; # The rate at which I becomes R
    s=1/L_days; # rate at which E becomes I
    g=1/I_days; # rate at which I becomes R
    
    if Rt[group] == L:
      fun = (n_groups)*Rt[group]*g*S*I.sum() + (rho*delta*(R_other-L)*g*S*I_other[:,np.newaxis]).sum() + (rho*delta**2*(R_other-L)*g*S*I.sum()).sum()
    else:
      fun = (n_groups*L + rho*(Rt[group]-L))*g*S*I.sum() + (rho*delta*(W-R_other)*g*S*I_other[:,np.newaxis]).sum()
    #We assume S is not changing
    dSdt[group] = 0
    dEdt[group,0] = fun-n_cont*s*E[0]
    for i in range(1,n_comp):
        dEdt[group,i] = n_cont*s*(E[i-1]-E[i])
    
    dIdt[group,0] = n_cont*(s*E[-1]-g*I[0])
    for i in range(1,n_comp):
        dIdt[group,i] = n_cont*s*(I[i-1]-I[i])

    dRdt[group]=np.array([n_cont*g*I[-1]]);
  
  dydt=np.concatenate([dEdt*dt,dIdt*dt,dSdt[:,np.newaxis]*dt,dRdt[:,np.newaxis]*dt],axis=1)
  return dydt

def test_time(t,period):
  te = lambda x: [(x>=y[0]) & (x<y[1]) for y in period]
  return any(te(t))


def start_sim(n_cont=2,R0=2.4,R_L=0.6,t_on=14,t_off=28,dt=0.1,all=False):
  # Start the simulation with a t_on days of free growth with R of R0 and the t_off days of lockdown with R_L
  n_groups = 1
  rh = 1
  R = np.ones(n_groups)
  delta = 0

  x_e = np.zeros((1,n_groups,n_cont*2+2))
  x_e[:,:,:2*n_cont] = 1e-3
  x_e[:,:,-2] = 1/n_groups

  # run for t_on days with unmitigated R
  Rt=R*R0
  for t in np.arange(0,t_on,dt):
      dx = SEIR_erlang(x_e[-1,:,:],n_groups,n_cont,Rt,L_days,I_days,delta,dt,rh,R0,R_L)
      x_e = np.append(x_e,(x_e[-1,:]+dx)[np.newaxis,:],axis=0)

  # run t_off weeks of lockdown
  Rt=R*R_L
  for t in np.arange(0,t_off,dt):
      dx = SEIR_erlang(x_e[-1,:,:],n_groups,n_cont,Rt,L_days,I_days,delta,dt,rh,R0,R_L)      
      x_e = np.append(x_e,(x_e[-1,:]+dx)[np.newaxis,:],axis=0)
  if all:
    return x_e.copy()
  else:
    return x_e[-1:,:,:]

def sim_groups(R_W,R_L,rh,t_cycle,dt,periods,n_groups,delta=0.1,plot=False,all=False,full=False,x0=None,tmax=125,with_fit=False):
  # simulate n groups with a work-lockdown schedule define in the variable periods
  R = np.ones(n_groups)
  
  # initialize epidemic
  if x0 is None:
    x_e = np.zeros((1,n_groups,n_cont*2+2))
    x_start = start_sim(all=all)
    x_e[:,:,:] = x_start[-1,:,:]
    x_e[:,:,-2] = x_e[:,:,-2]/n_groups
  else:
    x_e = x0
  
  Rt = R*R_L
  t_sim = np.arange(0,tmax,dt)
  for t in t_sim:
    # set the state (work/lockdown) for each group
    for group in range(n_groups):
      if test_time(np.mod(t,t_cycle),periods[group]):
        Rt[group]=R_W
      else:
        Rt[group]=R_L      
    
    # run a simulation timestep
    dx = SEIR_erlang(x_e[-1,:,:],n_groups,n_cont,Rt,L_days,I_days,delta,dt,rh,R_W,R_L)
    x_e = np.append(x_e,(x_e[-1,:]+dx)[np.newaxis,:],axis=0)
  

  x_e_plot = x_e[:,:,n_cont:2*n_cont].sum(axis=(1,2))/x_e[0,:,n_cont:2*n_cont].sum()
  x_t = np.arange(0,x_e_plot.shape[0]*dt,dt)
  
  # fit a linear model to the log transformed time-series of the infected population
  linfit = LinearRegression()  
  linfit.fit((x_t).reshape(-1,1), np.log(x_e_plot.reshape(-1,1))) 
  
  if plot:
    if any([all,full]):
      if x0 is None:
        x_all = np.zeros([x_start.shape[0]+x_e.shape[0],n_groups,x_start.shape[2]])
        for g in range(n_groups):
          x_all[:,g:g+1,:] = np.concatenate([x_start,x_e[:,g:g+1,:]],axis=0)
      else:
        x_all = x_e
      if full:
        x_all_plot = x_all
      else:
        x_all_plot = x_all[:,:,n_cont:2*n_cont].sum(axis=(2))/x_all[0,:,n_cont:2*n_cont].sum()
      x_t_all = np.arange(0,x_all_plot.shape[0]*dt,dt)
      if with_fit:
        return linfit, x_e_plot[0], x_t_all,x_all_plot
      return x_t_all,x_all_plot
    else:
      return linfit,x_t,x_e_plot
  else:
    return linfit.coef_

def sim_bang(R_W,R_L,rh,t_off,dt,n_groups,delta=0.1,t_on=14,full=False):
  # simulate a upper and lower threshold for staring and stopping lockdown with n groups
  Rt = np.ones(n_groups)
  
  # initialize epidemic
  x_bang = start_sim(all=True,t_off=t_off)
  x_bang[:,:,-2] = x_bang[:,:,-2]/n_groups

  l_time = []
  w_time = []
  
  # set upper and lower thresholds
  c_t = 2
  c_l = x_bang[-1,:,n_cont:2*n_cont].sum()/x_bang[0,:,n_cont:2*n_cont].sum()*1.4

  Rt[:]=R_W
  
  # run simulation for a total of 168 days
  t_sim_norm = np.arange(0,168-t_on-t_off,dt)
  for t in t_sim_norm:
    # if the number of infected people has crossed the threshold, start locdown
    if x_bang[-1,:,n_cont:2*n_cont].sum()/x_bang[0,:,n_cont:2*n_cont].sum() >= c_t:
      if (Rt[:]==R_W).all():
          l_time.append(t)
      Rt[:]=R_L
    # if the number of infected people has decreased below the lower threshold, release lockdown
    elif x_bang[-1,:,n_cont:2*n_cont].sum()/x_bang[0,:,n_cont:2*n_cont].sum() <= c_l:
      if (Rt[:]==R_L).all():
          w_time.append(t)
      Rt[:]=R_W  
    
    # run a simulation timestep
    dx = SEIR_erlang(x_bang[-1,:,:],n_groups,n_cont,Rt,L_days,I_days,delta,dt,rh,R_W,R_L)
    x_bang = np.append(x_bang,(x_bang[-1,:]+dx)[np.newaxis,:],axis=0)
  
  if full:
    x_e_plot = x_bang
  else:
    x_e_plot = x_bang[:,:,n_cont:2*n_cont].sum(axis=(1,2))/x_bang[0,:,n_cont:2*n_cont].sum()
  x_t = np.arange(0,x_e_plot.shape[0]*dt-dt,dt)
  return x_t,x_e_plot,l_time,w_time
  
def plot_results(R_W,R_L,w_days,t_cycle):
  dt = 0.1
  nf= int(1/dt)
  t_on = 14
  t_off =28

#   w_days,t_cycle = cycle_days
  schedule = [[[0,w_days]]]
  fit,x_0,x_t_one,x_one_plot = sim_groups(R_W=R_W,R_L=R_L,delta=0,rh=1,t_cycle=t_cycle,dt=0.1,periods=schedule,n_groups=1,plot=True,all=True,with_fit=True)
  fig = plt.figure(figsize=(6,4))
  # plot data
  plt.plot(x_t_one,x_one_plot,color = 'orangered',linewidth=2)
  
  # plot fit
  t_fit = x_t_one[(t_on+t_off)*nf:]
  y_fit = np.exp(x_t_one[(t_on+t_off)*nf:]*fit.coef_+fit.intercept_).squeeze()
  ratio = x_one_plot[(t_on+t_off)*nf]/y_fit[0]
  y_fit = y_fit*ratio
  Re = '{:.2f}'.format(np.exp(fit.coef_*5)[0][0])
  plt.plot(t_fit,y_fit,'--k',linewidth=2)

  # plot shaded areas for work/lockdown
  plt.axvspan(t_on, t_on+t_off, alpha=0.5, color='grey')
  for t in np.arange((t_on+t_off),x_t_one[-1],t_cycle):
      plt.axvspan(t+w_days, t+t_cycle, alpha=0.5, color='grey')
      plt.axvspan(t, t+w_days, alpha=0.2, color='grey')

  # plot formatting
  plt.xlabel('weeks',fontsize=14)
  plt.ylabel('number of cases',fontsize=14)
  plt.title(f'{w_days}-{t_cycle-w_days} cyclic lockdown strategy',fontsize=14,pad=40,bbox=dict(facecolor='#fde2be', alpha=0.5,edgecolor='none'))
  plt.text(43,4.2,f'resulting in $R_t$ of: '+ r"$\bf{" + str(Re) + "}$",fontsize=14,bbox=dict(facecolor='#fde2be', alpha=0.5,edgecolor='none'))
  plt.ylim([0,4])
  plt.xticks(np.arange(0,x_one_plot.shape[0],14),labels=np.arange(-2,x_one_plot.shape[0]/7-2,2).astype(int),fontsize=14)
  plt.xlim([2*7,18*7])
#   plt.text(16,0.3,f"$R_t$ = {Re}",fontsize=14,color='k')
  plt.yticks([]);

## <center>SEIR-Model with cyclic lockdown strategy</center>

This is an SEIR-Erlang model demostrating the results of a cyclic lockdown strategy with 4 days of work and 10 days of lockdown, with 14 day cycle. The effective reproduction numbers in work days $R_W$ as well and the reproduction number in lockdown days $R_L$ could by tuned, as well as the number of work days and the total cycle length. The simulation caclulates an effective R for the population $R_t$. 

In [82]:
slider_layout = widgets.Layout( height='160px')
RW = widgets.FloatSlider(value=1.5,min=1,max=3,step=0.1,orientation='Vertical',readout_format='.1f',layout=slider_layout,description='R_W')
# RW = v.Slider(value=1.5,thumb_label='always',vertical=True,style_='height: 75px, width: 80px')
RL = widgets.FloatSlider(value=0.6,min=0,max=1,step=0.1,orientation='Vertical',readout_format='.1f',layout=slider_layout,description='R_L')
slider_layout2 = widgets.Layout(width='200px')
# d = widgets.IntRangeSlider(value=[4, 14],min=0,max=14,step=1,readout=False)#,disabled=False,continuous_update=True,orientation='horizontal',readout=True,readout_format='d')
cycle = widgets.IntSlider(value=14,min=1,max=14,layout=slider_layout2)
work = widgets.IntSlider(value=4,min=1,max=14,layout=slider_layout2)
widgets.link((cycle,'value'),(work,'max'))

out = widgets.interactive_output(plot_results, {'R_W':RW,'R_L':RL,'w_days':work,'t_cycle':cycle})

labels = [widgets.HTMLMath(r'$R_W$'),widgets.HTMLMath(r'$R_L$')]
# widgets.HBox([widgets.VBox(labels),widgets.VBox([RW,RL,d])])labels = []
buttons = [RW,RL]
control_panel = widgets.HBox(buttons)
# control_panel = widgets.HBox([widgets.VBox([x,y],align_content='center') for x,y in zip(labels,buttons)])
labels2 = [widgets.Label(r'Work days'),widgets.Label(r'Total cycle days')]

days_control = widgets.HBox([widgets.VBox([x,y]) for x,y in zip(labels2,[work,cycle])])
main = widgets.VBox([out,days_control])
main.layout.justify = 'center'
widgets.HBox([control_panel,main])

For full description of the work please see:
* [medRxiv preprint](https://www.medrxiv.org/content/10.1101/2020.04.04.20053579v2)
* [medium post](https://medium.com/@urialonw/adaptive-cyclic-exit-strategies-from-lockdown-to-suppress-covid-19-and-allow-economic-activity-4900a86b37c7) explainin the work
* [fanancial times](https://www.ft.com/content/5c208540-831c-11ea-b6e9-a94cffd1d9bf) article discussing the suggeted strategy
* [One page summary](https://docs.google.com/document/d/1uWimqAgOf0624IWkwCzi4e1VFU7OOpKXeZ4UncfaEOU/edit?usp=sharing) for policy makers
* [stochastic simulation](https://github.com/omerka-weizmann/2_day_workweek/blob/master/code.ipynb)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1HmYVBzxf68rnB1z_LeSBHNHfffrrvcna) 
